In [1]:
# REQUIREMENT: Install GDAL 
#     * sudo apt install libgdal-dev
#     * pip install pygdal=="`gdal-config --version`.*"
from osgeo import ogr, osr

prj_text = 'PROJCS["NAD_1983_USFS_R6_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",43.0],PARAMETER["Standard_Parallel_2",48.0],PARAMETER["Latitude_Of_Origin",34.0],UNIT["Meter",1.0]]'
e_north = 1569101.1592
e_west = 510820.7258

ctr_srid = 4326
ctr_lat = 47.713149
ctr_lon = -120.7878

srs = osr.SpatialReference()
srs.ImportFromWkt(prj_text)

wgs84 = osr.SpatialReference()
wgs84.ImportFromEPSG(ctr_srid)

#to_srid = osr.CoordinateTransformation(wgs84, srs)
#to_wgs84 = osr.CoordinateTransformation(srs, wgs84)

nw_corner = ogr.CreateGeometryFromWkt('POINT (%f %f)' % (e_west, e_north))
centroid = ogr.CreateGeometryFromWkt('POINT (%f %f)' % (ctr_lon, ctr_lat))
#centroid.Transform(to_srid)

print("NW Corner: %s" % nw_corner.ExportToWkt())
print("Centroid: %s" % centroid.ExportToWkt())

NW Corner: POINT (510820.7258 1569101.1592)
Centroid: POINT (-120.7878 47.713149)


In [2]:
num_rows = 1123
num_cols = 729
cellsize = 90
e_south = e_north - cellsize*num_rows
sw_corner = ogr.CreateGeometryFromWkt('POINT (%f %f)' % (e_west, e_south))
print(e_south)
print(sw_corner.ExportToWkt())


1468031.1592
POINT (510820.7258 1468031.1592)


In [3]:
# Requires pyproj, installable via pip
from pyproj import CRS, Transformer
wena_crs = CRS.from_wkt('PROJCS["NAD_1983_USFS_R6_Albers",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Albers"],PARAMETER["False_Easting",600000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-120.0],PARAMETER["Standard_Parallel_1",43.0],PARAMETER["Standard_Parallel_2",48.0],PARAMETER["Latitude_Of_Origin",34.0],UNIT["Meter",1.0]]')
crs_4326 = CRS.from_epsg(4326)
transformer = Transformer.from_crs(wena_crs, crs_4326)
(e_north_4326, e_west_4326) = transformer.transform(e_west, e_north)
(e_south_4326, e_west_4326) = transformer.transform(e_west, e_south)

nw_corner_4326 = ogr.CreateGeometryFromWkt('POINT (%f %f)' % (e_west_4326, e_north_4326))
sw_corner_4326 = ogr.CreateGeometryFromWkt('POINT (%f %f)' % (e_west_4326, e_south_4326))

wgs_dist = nw_corner_4326.Distance(sw_corner_4326)
pnnl_dist = nw_corner.Distance(sw_corner)
print("Wgs distance: %f" % wgs_dist)
print("Pnnl distance: %f" % pnnl_dist)
print(cellsize*num_rows)

Wgs distance: 0.908752
Pnnl distance: 101070.000000
101070


In [ ]:
# see https://gis.stackexchange.com/a/328642/53225 for shapely soln

# TODO: pull a basin from a shapefile, transform to the custom PNNL CRS...
#   Maybe loop through all basins in the shapefile, maybe pull one out by stream ID
#   Final soln may be pre-made mask.asc files instead of shapefile, named by Stream ID
#       This would use reading features into rasterio, then burning them into a raster
#           https://rasterio.readthedocs.io/en/latest/topics/features.html#burning-shapes-into-a-raster

In [ ]:
# TODO: read in Wena 2020 mask
#   Clip it to the selected feature from the last section.
#   Exported clip should be an asc with the same extent and NODATA value